<a href="https://colab.research.google.com/github/GhislainBisamaza/ParentDocumentRetriever/blob/main/ParentDocumentRetriever_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain
!pip install langchain_chroma
!pip install langchain_openai
!pip install langchain_community
!pip install langchain_text_splitters
!pip install mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install --upgrade langchain

In [10]:
import os
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from mistralai.client import MistralClient
# from langchain.vectorstores import Mistral

In [22]:
# loaders = [
#     TextLoader("/sample1_Computer_vision.pdf", encoding='latin-1'), # Try a different encoding like 'latin-1'
#     TextLoader("/sample2_LLM.pdf", encoding='latin-1'), # Apply the same encoding to the second file as well
# ]
# docs = []
# for loader in loaders:
#     docs.extend(loader.load())

loaders = [
    TextLoader("/paul_graham_essay.txt"),
    TextLoader("/state_of_the_union.txt"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [23]:
docs

[Document(metadata={'source': '/paul_graham_essay.txt'}, page_content="Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep."),
 Document(metadata={'source': '/state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.')]

In [42]:
# # This text splitter is used to create the child documents
# child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# # The vectorstore to use to index the child chunks


# client = MistralClient(api_key="PZ6WfVj5evbmcwz7D2xPDUrjs3rEMjGb")

# class MistralEmbeddings:
#     def __init__(self, client):
#         self.client = client

#     def embed_documents(self, texts):
#         embeddings_response = self.client.embeddings(model="mistral-embed", input=texts)
#         return [embedding.embedding for embedding in embeddings_response.data]

# # The vectorstore to use to index the child chunks
# vectorstore = Chroma(
#   collection_name="full_documents",
#   embedding_function=MistralEmbeddings(client) # Pass an instance of the class
# )

# # The storage layer for the parent documents
# store = InMemoryStore()
# retriever = ParentDocumentRetriever(
#     vectorstore=vectorstore,
#     docstore=store,
#     child_splitter=child_splitter,
# )

In [49]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks


client = MistralClient(api_key="PZ6WfVj5evbmcwz7D2xPDUrjs3rEMjGb")

class MistralAIEmbeddings:
    def __init__(self, client):
        self.client = client

    def embed_documents(self, texts):
        embeddings_batch_response = self.client.embeddings(
            model="mistral-embed",
            input=texts,
        )
        # Access the embedding attribute of each EmbeddingObject
        return [embedding.embedding for embedding in embeddings_batch_response.data]

    def embed_query(self, text):
        # Embed a single query text
        embedding_response = self.client.embeddings(model="mistral-embed", input=[text])
        # Return the embedding for the query
        return embedding_response.data[0].embedding # Access the embedding attribute

# Initialize the text splitter for creating child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# Initialize the vector store with the Mistral embedding function
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=MistralAIEmbeddings(client)  # Pass the MistralAIEmbeddings instance
)

# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [50]:
retriever.add_documents(docs, ids=None)

In [51]:
list(store.yield_keys())

['7eb7857b-bb88-4fcf-b560-92c3c9c41047',
 '385e9364-c4b3-4103-91d7-c6acfb43b611']

In [57]:
sub_docs = vectorstore.similarity_search("writing and programming")

In [58]:
print(sub_docs[0].page_content)
retrieved_docs = retriever.invoke("writing and programming")

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.


In [59]:
len(retrieved_docs[0].page_content)

338